<a href="https://colab.research.google.com/github/Baco23/github-course/blob/master/LSTM_Quina1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

Prepare/Generate data set //
let´s imagine that we have history of lottery by generating 80 numbers


In [2]:
df = pd.DataFrame(np.random.randint(1,80,size=(80, 5)), columns=list('ABCDE'))

The top games look like this (A B C D E F are balls that take range 1 to 60 each):

In [3]:
df.head()

,A,B,C,D,E
0,4,7,67,76,34
1,55,58,31,61,73
2,4,17,72,20,6
3,76,58,40,66,42
4,5,3,9,32,57


Next we need to normalize data.

In [4]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

Lets define hyper params of or model

In [5]:
number_of_rows = df.values.shape[0] #all our games
window_length = 68 #amount of past games we need to take in consideration for prediction
number_of_features = df.values.shape[1]

Create train dataset and labels for each row. It should have format for keras lstm model(row, window sixe, balls)

In [6]:
train = np.empty([number_of_rows-window_length, window_length, number_of_features], dtype=float)

label = np.empty([number_of_rows-window_length, number_of_features], dtype=float)

window_length = 68

for i in range(0, number_of_rows-window_length):
  train[i]=transformed_df.iloc[i:i+window_length, 0: number_of_features]
  label[i]=transformed_df.iloc[i+window_length: i+window_length+1, 0:number_of_features]

Shapes

In [7]:
train.shape

(12, 68, 5)

In [8]:
label.shape

(12, 5)

In [9]:
train[0]

array([[-1.37859692, -1.25833708,  1.00792307,  1.47321573, -0.25283286],
       [ 0.76904548,  0.8363244 , -0.50553806,  0.81917989,  1.46952197],
       [-1.37859692, -0.84761914,  1.218126  , -0.96851807, -1.48939529],
       [ 1.65336882,  0.8363244 , -0.12717277,  1.03719184,  0.1004707 ],
       [-1.33648629, -1.42262426, -1.43043096, -0.4452894 ,  0.76291486],
       [ 1.65336882, -0.68333197,  0.67159837,  1.12439662, -1.57772118],
       [ 0.72693484,  0.54882184,  0.16711133,  1.38601095, -1.35690646],
       [ 0.30582849, -0.43690121, -0.33737571, -0.31448223, -1.18025468],
       [ 0.43216039, -1.34048067,  1.00792307, -0.18367507,  0.67458897],
       [-0.49427358,  0.46667826,  0.75567955,  1.03719184,  0.32128542],
       [ 1.73759009,  0.54882184, -1.26226862,  0.47036078,  1.42535902],
       [ 0.81115611, -0.88869094,  0.67159837, -1.79696347,  1.64617374],
       [-0.2837204 ,  1.12382696,  1.218126  , -0.31448223,  1.51368491],
       [ 0.68482421, -0.92976273,  0.1

In [10]:
train[1]

array([[ 0.76904548,  0.8363244 , -0.50553806,  0.81917989,  1.46952197],
       [-1.37859692, -0.84761914,  1.218126  , -0.96851807, -1.48939529],
       [ 1.65336882,  0.8363244 , -0.12717277,  1.03719184,  0.1004707 ],
       [-1.33648629, -1.42262426, -1.43043096, -0.4452894 ,  0.76291486],
       [ 1.65336882, -0.68333197,  0.67159837,  1.12439662, -1.57772118],
       [ 0.72693484,  0.54882184,  0.16711133,  1.38601095, -1.35690646],
       [ 0.30582849, -0.43690121, -0.33737571, -0.31448223, -1.18025468],
       [ 0.43216039, -1.34048067,  1.00792307, -0.18367507,  0.67458897],
       [-0.49427358,  0.46667826,  0.75567955,  1.03719184,  0.32128542],
       [ 1.73759009,  0.54882184, -1.26226862,  0.47036078,  1.42535902],
       [ 0.81115611, -0.88869094,  0.67159837, -1.79696347,  1.64617374],
       [-0.2837204 ,  1.12382696,  1.218126  , -0.31448223,  1.51368491],
       [ 0.68482421, -0.92976273,  0.12507075,  1.6040229 , -0.47364758],
       [-0.45216295, -1.34048067, -0.8

In [11]:
label[0]

array([ 1.02170929,  0.26131929,  1.47036952, -0.88131329, -1.66604707])

In [12]:
label[1]

array([-0.57849485, -0.68333197,  1.42832893, -0.57609657, -1.0036029 ])

Our LSTM model

In [13]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import numpy as np

batch_size = 20

Using TensorFlow backend.


Using TensorFlow backend.

In [14]:
model = Sequential()
model.add(LSTM(80,
               input_shape=(window_length, number_of_features),
               return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(80,
               return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(number_of_features))
model.compile(loss='mse', optimizer='rmsprop')

Training

In [15]:
model.fit(train, label,
          batch_size=64, epochs=3000)

Streaming output truncated to the last 5000 lines.
12/12 [==============================] - 0s 7ms/step - loss: 0.0358
Epoch 502/3000
12/12 [==============================] - 0s 7ms/step - loss: 0.0523
Epoch 503/3000
12/12 [==============================] - 0s 7ms/step - loss: 0.0270
Epoch 504/3000
12/12 [==============================] - 0s 7ms/step - loss: 0.0269
Epoch 505/3000
12/12 [==============================] - 0s 7ms/step - loss: 0.0476
Epoch 506/3000
12/12 [==============================] - 0s 8ms/step - loss: 0.0289
Epoch 507/3000
12/12 [==============================] - 0s 8ms/step - loss: 0.0316
Epoch 508/3000
12/12 [==============================] - 0s 7ms/step - loss: 0.0303
Epoch 509/3000
12/12 [==============================] - 0s 7ms/step - loss: 0.0365
Epoch 510/3000
12/12 [==============================] - 0s 7ms/step - loss: 0.0415
Epoch 511/3000
12/12 [==============================] - 0s 8ms/step - loss: 0.0518
Epoch 512/3000
12/12 [=============================

Prediction, lets imagene we know last 60 games and want to predict next game.

In [18]:
to_predict=np.array([[49,34,7,14,63],
[61,59,65,22,26],
[23,32,2,38,49],
[79,12,16,39,24],
[73,18,25,78,8],
[62,13,38,51,74],
[14,49,30,76,75],
[37,79,41,67,34],
[4,29,46,34,73],
[5,41,8,79,38],
[52,28,36,14,60],
[31,33,63,59,22],
[15,3,52,46,75],
[72,66,46,2,69],
[8,69,23,12,65],
[63,50,71,60,7],
[61,16,60,21,53],
[39,55,67,1,54],
[70,10,2,11,50],
[60,50,40,21,13],
[18,29,71,4,44],
[4,68,30,21,49],
[74,24,3,7,65],
[58,26,47,14,70],
[51,43,11,73,15],
[33,25,74,57,21],
[79,77,26,38,41],
[57,11,44,70,4],
[78,49,70,57,29],
[7,8,33,23,1],
[20,53,2,30,27],
[60,69,20,34,66],
[4,79,80,12,26],
[10,3,38,22,45],
[34,73,17,23,26],
[58,18,10,72,4],
[60,59,39,16,40],
[33,15,11,21,57],
[8,46,27,54,34],
[19,62,61,31,71],
[22,52,21,34,27],
[73,68,65,66,14],
[16,60,26,13,17],
[20,78,43,22,25],
[62,38,15,26,16],
[29,14,39,76,54],
[33,77,48,3,21],
[63,6,35,24,48],
[47,9,4,53,62],
[48,49,1,63,52],
[71,68,24,14,54],
[35,20,71,36,77],
[40,68,12,38,53],
[12,79,65,10,29],
[25,51,12,60,74],
[36,71,44,75,33],
[44,67,14,10,30],
[32,6,24,26,29],
[1,38,71,9,30],
[56,62,78,70,10],
[67,65,71,47,46],
[37,28,67,12,39],
[18,20,38,2,23],
[49,27,25,73,6],
[78,8,44,64,66  ],                    
[59,39,52,43,7],
[75,4,78,65,43],
[46,76,60,47,51]])
scaled_to_predict = scaler.transform(to_predict) 

In [20]:
scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
print(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])

[17 23 42 27 60]
